# Оценка результатов A/B теста

1. [Описание и цели](#objectives)
2. [Обзор и предобработка данных](#preprocessing)
3. [Изучение результатов эксперимента](#results)
    * [A/A тест](#aatest)
    * [Функция сравнения статистических различий долей выборок](#def)
    * [A/B тест](#abtest)
8. [Выводы](#summary)

## Описание и цели<a id="objectives"></a>

На сайте внедрили новое решение — назойливый чат с менеджером.

Необходимо провести анализ результатов A/B теста для выборок пользователей, которым показывали чат и тем, кто пользовался версией сайта без чата.

Определить есть ли различия между выборками 2х контрольных групп.

## Обзор и предобработка данных<a id="preprocessing"></a>

In [1]:
# импорт бибилиотек
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import plotly.graph_objects as go
from scipy import stats as st
import math as mth

In [2]:
#чтение файлов
path = ''
data = pd.read_csv(path+'ab_log.csv')

In [3]:
data.head()

,event,user_id,EventTime,Group
0,StartReadingOffer,7241586465170049200,1564978321,A2
1,OpenProductCard,4998498972125515821,1564678223,A2
2,StartReadingOffer,5794343176311717876,1564681595,B
3,StartReadingOffer,3371640160394640961,1564635919,A1
4,OpenProductCard,4623191541226589455,1564695060,A1


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154788 entries, 0 to 154787
Data columns (total 4 columns):
event        154788 non-null object
user_id      154788 non-null int64
EventTime    154788 non-null int64
Group        154788 non-null object
dtypes: int64(2), object(2)
memory usage: 4.7+ MB


In [5]:
#преобразуем дату
data['EventTime'] = pd.to_datetime(data['EventTime'], unit='s')

In [6]:
#проверяем пропущенные значения
data.isnull().sum()

event        0
user_id      0
EventTime    0
Group        0
dtype: int64

**Вывод:** Проверили датасет - пропусков не найдено. Преобразовали колонку времени события в тип дата-время. 

## Изучение результатов эксперимента<a id="results"></a>

In [7]:
#число пользователей в каждой контрольной группе
data.groupby('Group')['user_id'].nunique()

Group
A1    2276
A2    2236
B     2293
Name: user_id, dtype: int64

Есть 2 контрольные группы для А/А-эксперимента, чтобы проверить корректность всех механизмов и расчётов, проверем, находят ли статистические критерии разницу между выборками А1 и А2.

In [8]:
#сформируем выборки по 2м контрольным группам

#экспериментальная группа А1

sampleA1_ungrouped = data.query('Group == "A1"')

sampleA1 = sampleA1_ungrouped.groupby('event').agg({'user_id': 'nunique'}).sort_values(by='user_id', ascending=False).reset_index()
sampleA1['ratio'] = sampleA1['user_id'] / sampleA1_ungrouped['user_id'].nunique()
sampleA1

,event,user_id,ratio
0,StartReadingOffer,2223,0.976714
1,StartCheckProduct,1330,0.584359
2,OpenPaymentScreen,1117,0.490773
3,OpenProductCard,1077,0.473199


In [9]:
#экспериментальная группа А2

sampleA2_ungrouped = data.query('Group == "A2"')

sampleA2 = sampleA2_ungrouped.groupby('event').agg({'user_id': 'nunique'}).sort_values(by='user_id', ascending=False).reset_index()
sampleA2['ratio'] = sampleA2['user_id'] / sampleA2_ungrouped['user_id'].nunique()
sampleA2

,event,user_id,ratio
0,StartReadingOffer,2185,0.977191
1,StartCheckProduct,1343,0.600626
2,OpenPaymentScreen,1142,0.510733
3,OpenProductCard,1096,0.490161


### Проверка А/А теста<a id="aatest"></a>

In [10]:
#сравниваем группы по размеру

size_A1 = sampleA1_ungrouped['user_id'].nunique()
size_A2 = sampleA2_ungrouped['user_id'].nunique()

print('Контрольные группы отличаются на', round((size_A1-size_A2)/size_A1 *100, 2), '%' )

Контрольные группы отличаются на 1.76 %


К сожалению, размер контрольных групп отличается более, чем на 1%. В идеале, такой А/А тест следует считать провальным, но считаем что разница все таки около 1% и продолжим исследование.

1. Проверим, есть ли статистическая разница между контрольными выборками A1 и A2 по средним значениям распределения пользователей по событиям.

Нулевая гипотеза: выборки равноценны.

In [11]:
#проверяем по кол-ву пользователей

sample_1 = sampleA1['user_id']

sample_2 = sampleA2['user_id']

alpha = .05 # критический уровень статистической значимости

results = st.ttest_ind(
    sample_1, 
    sample_2)

print('p-значение:', results.pvalue)

if (results.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

p-значение: 0.990143378389625
Не получилось отвергнуть нулевую гипотезу


2. Проверим, есть ли статистическая разница между контрольными выборками A1 и A2 по средним значениям долей.

Нулевая гипотеза: выборки равны.

In [12]:
#проверяем по кол-ву пользователей и по долям.

sample_1 = sampleA1['ratio']

sample_2 = sampleA1['ratio']

alpha = .05 # критический уровень статистической значимости

results = st.ttest_ind(
    sample_1, 
    sample_2)

print('p-значение:', results.pvalue)

if (results.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

p-значение: 1.0
Не получилось отвергнуть нулевую гипотезу


**Вывод:** A/A тест прошел успешно. Не смотря на то, что размер выборок больше 1%, статистические критерии не находят разницу между выборками контрольных групп A1 и A2.

### Функция определения статистического различия долей выборок по событию<a id="def"></a>

In [13]:
#функция определения статистического различия долей выборок по событию

def stat_difference(event, sample1, sample2):
    alpha = .05
    
    success1 = sample1.query('event == @event').groupby('event')['user_id'].nunique().sum() 
    success2 = sample2.query('event == @event').groupby('event')['user_id'].nunique().sum() 

    
    trial1 = sample1['user_id'].nunique()
    trial2 = sample2['user_id'].nunique()
    

    # пропорция успехов в первой группе:
    p1 = success1/trial1

    # пропорция успехов во второй группе:
    p2 = success2/trial2

    # пропорция успехов в комбинированном датасете:
    p_combined = (success1 + success2) / (trial1 + trial2)

    # разница пропорций в датасетах
    difference = p1 - p2
    print('Разница пропорций:', difference)

    z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1/trial1 + 1/trial2))

    # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
    distr = st.norm(0, 1) 

    p_value = (1 - distr.cdf(abs(z_value))) * 2

    print('p-значение: ', p_value)

    if (p_value < alpha):
        print("Отвергаем нулевую гипотезу: между долями есть значимая разница")
    else:
        print("Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными")

### A/B тест<a id="abtest"></a>

**Сравнение контрольных групп A1 и A2 с группой с добавленным чатом (группа B)**

In [14]:
#все события, о которых есть записи в датасете
all_events = data['event'].unique()

In [15]:
#выборка данных по группе B
sampleB = data.query('Group == "B"')

In [16]:
#сравнение стат значимости различий долей групп A1 и B по каждому событию

for event in all_events:
    print('\033[1m' + 'Проверка стат различий по событию:', event, '\033[0m')
    stat_difference(event, sampleA1_ungrouped, sampleB)
    print('')

Проверка стат различий по событию: StartReadingOffer 
Разница пропорций: -0.003661522000556361
p-значение:  0.3930100442290123
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными

Проверка стат различий по событию: OpenProductCard 
Разница пропорций: -0.0012889768432541304
p-значение:  0.930475633720478
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными

Проверка стат различий по событию: StartCheckProduct 
Разница пропорций: 0.006949016530021446
p-значение:  0.6340897766832789
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными

Проверка стат различий по событию: OpenPaymentScreen 
Разница пропорций: 0.0084357374051231
p-значение:  0.5683988278934891
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными



In [17]:
#сравнение стат значимости различий долей групп A2 и B по каждому событию

for event in all_events:
    print('\033[1m' + 'Проверка стат различий по событию:', event, '\033[0m')
    stat_difference(event, sampleA2_ungrouped, sampleB)
    print('')

Проверка стат различий по событию: StartReadingOffer 
Разница пропорций: -0.003183641275812499
p-значение:  0.4570777288860348
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными

Проверка стат различий по событию: OpenProductCard 
Разница пропорций: 0.015673430921050058
p-значение:  0.2912553866611711
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными

Проверка стат различий по событию: StartCheckProduct 
Разница пропорций: 0.0232166108721652
p-значение:  0.1123805625336558
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными

Проверка стат различий по событию: OpenPaymentScreen 
Разница пропорций: 0.028395903531553968
p-значение:  0.056019682368946055
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными



In [18]:
#сравнение объединенной группы A1 с A2 и группы B

#объединенная выборка A1 и A2
sampleA1_A2 = data.query('Group == "A1" | Group == "A2"')

for event in all_events:
    print('\033[1m' + 'Проверка стат различий по событию:', event, '\033[0m')
    stat_difference(event, sampleA1_A2, sampleB)
    print('')

Проверка стат различий по событию: StartReadingOffer 
Разница пропорций: -0.003424699903808337
p-значение:  0.3615160775555841
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными

Проверка стат различий по событию: OpenProductCard 
Разница пропорций: 0.007117039061219299
p-значение:  0.5785582485744634
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными

Проверка стат различий по событию: StartCheckProduct 
Разница пропорций: 0.015010705570144167
p-значение:  0.23448894151684985
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными

Проверка стат различий по событию: OpenPaymentScreen 
Разница пропорций: 0.018327344554657565
p-значение:  0.15291326723344678
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными



# Выводы<a id="summary"></a>

А/B тест показал, что явных различий между конверсией контрольных групп и группы с добавленным чатом не обнаружено. Но при установленном уровне значимости 0,1, отвергается гипотеза о равенстве выборок в событии "OpenPaymentScreen" при сравнении A2 и B групп.

Но при сравнении группы B группы с A1 и с объединенной группой (A1 и A2) при обоих значениях уровня значимости, выборки долей считаются равноценными. Поэтому считаем правильным установить уровень занчимости: 0.05. 

В целом добавление чата на сайт не влияет на конверсию.